# Tracking

There are various tracking library we can use to link object over time in order to define trajectories.


## Bayesian tracker 

The first tracker that we will test is the BayesianTracker https://github.com/quantumjot/BayesianTracker .

Reference:
Automated deep lineage tree analysis using a Bayesian single cell tracking approach, Ulicna K, Vallardi G, Charras G and Lowe AR., bioRxiv (2020), https://www.biorxiv.org/content/early/2020/09/10/2020.09.10.276980

Installation
We can install the package using ```pip install btrack```

Let's try a first example

In [ ]:
import btrack
import numpy as np
import matplotlib.pyplot as plt

# We define a list of positions we want to track later
num_objects = 10 # number of objects
num_frame = 20 # number of time points
pts = [] # list of set of coordinates
for k in range(num_frame):
    if k==0:
        pts.append(np.hstack([np.random.rand(10,2), np.zeros((10,1))]))
    else:
        pts.append(pts[k-1]+np.hstack([0.01*np.random.randn(10,2), np.ones((10,1))]))
        
localizations = np.concatenate(pts)
plt.plot(localizations[:,0], localizations[:,1],'.')

In [ ]:
# Download a generic motion model to use with btrack
from urllib.request import urlretrieve
url = 'https://raw.githubusercontent.com/quantumjot/BayesianTracker/7d80399db64af4723c1da2381a22d9910e1ba7c8/models/cell_config.json'
config = '../scratch/cell_config.json'
urlretrieve(url, config)

In [ ]:
# Convert the numpy localization as PyTrackObjects
objects = btrack.dataio.objects_from_array(localizations, ['x', 'y', 't'])

with btrack.BayesianTracker() as tracker:
    # load the configuration file
    tracker.configure_from_file(config)
    # add the objects to the tracker
    tracker.append(objects)
    tracker.volume=((-0.5, 1.5), (-0.5, 1.5), (-1e5, 1e5))
    # tracl objects
    tracker.track_interactive(step_size=1)
    # run the global optimizer
    tracker.optimize()
    # tracks as a python list
    tracks = tracker.tracks
    # get tracks formated for napari
    data, properties, graph = tracker.to_napari(ndim=2)

To represent the tracks in 2D we can plot them individually.  tracker.tracks is a list of dictionnary containing various fields such as ID,x,y,z,t,parent,root,state,etc..

In [ ]:
for trck in tracks:
    plt.plot(trck.x,trck.y)

### Track visualization with napari
note: if you run this with python 3.9, you need to use an specific version of networkx to be ```conda install -c conda-forge networkx=2.5```

In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_tracks(data, properties=properties, graph=graph)

## Tracking with trackpy

http://soft-matter.github.io/trackpy/v0.5.0/
